In [10]:
import numpy as np
import tqdm
import operator
import lightgbm as lg
import os

In [11]:
#data 
queries = {}
with open('./features/norm_q.txt') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if line[0] == '':
            line.pop(0)
        queries[line[0]] = line[1]

titles= {}
with open("./features/norm_tits.txt" ,'r', encoding='utf-8') as fin:
    for line in fin.readlines():
        line=line.split('\t')
        titles[line[0]]= line[1][:-1]

train=[]
with open("./initial_data/train.marks.tsv/train.marks.tsv", 'r') as fin:
    for line in fin.readlines():
        line=line[:-1].split('\t')
        train.append([line[0],line[1], line[2]])


                
test=[]
with open("./initial_data/sample.csv/sample.csv", 'r') as fin:
    fin.readline()
    for line in fin.readlines():
        line=line[:-1].split(',')
        test.append([line[0],line[1],-1])

In [12]:
#ptfbm.py
texttf = {}
tffiles = os.listdir('./features/tf')
for file in tffiles:
    with open('./features/tf/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        texttf[(line[0], line[1])] = np.float64(line[2])

In [13]:
hostdict = {}
with open('./initial_data/url.data/url.data') as fin:
    lines = fin.readlines()
for line in lines:
    line = line[:-1].split('\t')
    if line[1][:7] == 'http://':
        line[1] = line[1][7:]
    if line[1][:4] == 'www.':
        line[1] = line[1][4:]
    hostdict[line[0]] = line[1].split('/')[0]

In [14]:
#all.java
hostfiles = os.listdir('./features/DDF')
hf = {}
qhf = {}
c = 0
for file in hostfiles:
    with open('./features/DDF/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        line.pop()
        if file[0] == 'Q':
            q = line.pop(0)
            h = line.pop(0)
            qhf[(q,h)] = np.array(line, dtype = np.float64)
            c+=1
        else:
            h = line.pop(0)
            hf[h] = np.array(line, dtype = np.float64)

In [15]:
#allurl.java
urlfiles = os.listdir('./features/UDDF')
uf = {}
quf = {}
for file in urlfiles:
    with open('./features/UDDF/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        line.pop()
        if file[1] == 'Q':
            q = line.pop(0)
            u = line.pop(0)
            quf[(q,u)] = np.array(line, dtype = np.float64)
        else:
            u = line.pop(0)
            uf[u] = np.array(line, dtype = np.float64)

In [16]:
#simdoc.py
similars = {}
with open('./features/oldsimdoc.txt', 'r') as fin:
    lines = fin.readlines()
    for line in lines:
        spl = line[:-2].split('\t')
        if len(spl) == 1:
            continue
        tmp = spl[1].split(' ')
        for i in range(13):
            tmp.pop()
        similars[spl[0]] = tmp

In [17]:
#simdoc.py
file =open("./features/dvs.txt" ,'r', encoding='utf-8')
dvs= {}
for line in file.readlines():
    l=line.split('\t')
    dvs[(l[0], l[1])]= np.array(l[2][:-1], dtype = np.float64)
file.close()

In [18]:
#ptfbm_titles.py
file =open("./features/tf23.txt" ,'r', encoding='utf-8')
ttit1= {}
for line in file.readlines():
    l=line.split('\t')
    ttit1[(l[0], l[1])]= np.array(l[2][:-1], dtype = np.float64)
file.close()

In [19]:
#ptfbm_titles.py
file =open("./features/tf39.txt" ,'r', encoding='utf-8')
ttit3= {}
for line in file.readlines():
    l=line.split('\t')
    ttit3[(l[0], l[1])]= np.array(l[2][:-1], dtype = np.float64)
file.close()

In [20]:
#DBN.java
file =open("./features/DBN/part-r-00000" ,'r', encoding='utf-8')
dbndoc = {}
for line in file.readlines():
    l=line[:-1].split('\t')
    k = l[0]
    l.pop(0)
    dbndoc[k]= np.array(l, dtype = np.float64)
file.close()

In [21]:
#ptfbm.py
ltf = {}
ltffiles = os.listdir('./features/ltf')
for file in ltffiles:
    with open('./features/ltf/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        ltf[(line[0], line[1])] = np.float64(line[2])

In [22]:
#allq.java
file =open("./features/QF/part-r-00000" ,'r', encoding='utf-8')
qf = {}
for line in file.readlines():
    l=line[:-1].split('\t')
    k = l[0]
    l.pop(0)
    qf[k]= np.array(l, dtype = np.float64)
file.close()

In [23]:
#pas.py
pas = {}
pasfiles = os.listdir('./features/pas')
for file in pasfiles:
    with open('./features/pas/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            pas[(file[:-4], line[0])] = np.log(1 + np.float64(line[1]))
#ptfbm.py
newbm = {}
newbmfiles = os.listdir('./features/newbm')
for file in newbmfiles:
    with open('./features/newbm/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            newbm[(file[:-4], line[0])] = np.log(1+ np.float64(line[1]))


In [24]:
#simdoc.py
muse = {}
musefiles = os.listdir('./features/muse/muse')
for file in musefiles:
    with open('./features/muse/muse/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            muse[(line[0], line[1])] = np.float64(line[2])

In [25]:
#simdoc.py
luse = {}
with open('./features/luse.txt') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        luse[(line[0], line[1])] = np.float64(line[2])

In [26]:
#simdoc.py
quse = {}
with open('./features/quse.txt') as fin:
    lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        quse[(line[0], line[1])] = np.float64(line[2])

In [27]:
#pasT.py
past = {}
pastfiles = os.listdir('./features/pasT')
for file in pastfiles:
    with open('./features/pasT/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            past[(file[:-4], line[0])] = np.log(1+ np.float64(line[1]))
            
#pas.py
pas3 = {}
pas3files = os.listdir('./features/pas3')
for file in pas3files:
    with open('./features/pas3/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            pas3[(file[:-4], line[0])] = np.log(1+ np.float64(line[1]))

In [28]:
#simdoc.py
trft = {}
trftfiles = os.listdir('./features/trft')
for file in pastfiles:
    with open('./features/trft/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            trft[(file[:-4], line[0])] = np.float64(line[1])

In [29]:
#simdoc.py
ft = {}
ftfiles = os.listdir('./features/ftsim/ftsim')
for file in pas3files:
    with open('./features/ftsim/ftsim/' + file) as fin:
        lines = fin.readlines()
    for line in lines:
        line = line[:-1].split('\t')
        if len(line) > 1:
            ft[(file[:-4], line[0])] = np.float64(line[1])

In [30]:
def getdata(cat):
    trainx = []
    trainy = []
    qid_train = []
    index = []
    for entry in tqdm.tqdm(cat):
        try:
            try:
                tempx = hf[hostdict[entry[1]]]
            except:
                temp = None
    #             for q in similars[entry[0]]:
    #                 if hostdict[entry[1]] in hf.keys():
    #                     temp = hf[hostdict[entry[1]]]
    #                     break
                if temp is None:
                    [temp] = [[0.0] * 13]
                tempx = temp
            try:
                tempx = np.hstack((tempx,qhf[(entry[0],hostdict[entry[1]])]))
            except:
                temp = None
                for q in similars[entry[0]]:
                    if (q,hostdict[entry[1]]) in qhf.keys():
                        temp = qhf[(q,hostdict[entry[1]])]
                        break
                if temp is None:
                    [temp] = [[0.0] * 43]
                tempx = np.hstack((tempx,temp))

            try:
                tempx = np.hstack((tempx,uf[entry[1]]))
            except:
                temp = None
    #             if hostdict[entry[1]] in hf.keys():
    #                 temp = hf[hostdict[entry[1]]]
                for q in similars[entry[0]]:
                    if entry[1] in uf.keys():
                        temp = uf[entry[1]]
                        break
                if temp is None:
                    [temp] = [[0.0] * 13]
                tempx = np.hstack((tempx,temp))

            try:
                tempx = np.hstack((tempx,quf[(entry[0],entry[1])]))
            except:
                temp = None
    #             if (entry[0],hostdict[entry[1]]) in qhf.keys():
    #                 temp = qhf[(entry[0],hostdict[entry[1]])]
                for q in similars[entry[0]]:
                    if (q,entry[1]) in quf.keys():
                        temp = quf[(q,entry[1])]
                        break
                if temp is None:
                    [temp] = [[0.0] * 43]
                tempx = np.hstack((tempx,temp))
            try:
                tempx = np.hstack((tempx,texttf[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,dvs[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,ttit1[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,ttit3[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))

            try:
                tempx = np.hstack((tempx,dbndoc[entry[1]]))
            except:
                temp = None
                if temp is None:
                    [temp] = [[0.0] * 3]
                tempx = np.hstack((tempx,temp))
            try:
                tempx = np.hstack((tempx,ltf[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))

            try:
                tempx = np.hstack((tempx,qf[entry[0]]))
            except:
                temp = None
                if temp is None:
                    for q in similars[entry[0]]:
                        if q in qf.keys():
                            temp = qf[q]
                            break
                if temp is None:
                    [temp] = [[0.0] * 23]
                tempx = np.hstack((tempx,temp))

            try:
                tempx = np.hstack((tempx,pas[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,newbm[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,trft[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,muse[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,past[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,pas3[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,ft[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            try:
                tempx = np.hstack((tempx,quse[(entry[0],entry[1])]))
            except:
                tempx = np.hstack((tempx,np.array([0.0])))
            trainx.append(tempx)
            trainy.append(float(entry[2]))
            qid_train.append(int(entry[0]))
            index.append((entry[0], entry[1]))
        except:
            continue
    Xtrain = np.row_stack([row for row in trainx])
    Ytrain = np.row_stack([row for row in trainy])
    return Xtrain, Ytrain, qid_train, index

In [47]:
Xtrain = np.load('./ready-to-use-features/Xtrain.npy')
Ytrain = np.load('./ready-to-use-features/Ytrain.npy')
Xtest = np.load('./ready-to-use-features/Xtest.npy')
qqg = np.load('./ready-to-use-features/qqg.npy')

In [33]:
Xtrain, Ytrain, qid_train, tindex = getdata(train)

100%|██████████| 202079/202079 [00:31<00:00, 6421.21it/s]


In [34]:
qqg = np.unique(qid_train, return_counts=True)[1]

In [35]:
Xtest,Ytest, _ , index = getdata(test)

100%|██████████| 403971/403971 [01:10<00:00, 5760.50it/s]


In [39]:
lgrank = lg.LGBMRanker(objective='lambdarank', max_depth=12, n_estimators= 1000, subsample=0.8, learning_rate=0.01, num_leaves=64)

In [40]:
%%time
lgrank.fit(Xtrain,Ytrain.ravel(),group=qqg)

CPU times: user 25min 25s, sys: 6.1 s, total: 25min 31s
Wall time: 4min 2s


LGBMRanker(learning_rate=0.01, max_depth=12, n_estimators=1000, num_leaves=64,
           objective='lambdarank', subsample=0.8)

In [41]:
%%time
cres = lgrank.predict(Xtest)

CPU times: user 1min 10s, sys: 104 ms, total: 1min 10s
Wall time: 10.9 s


In [42]:
lgrank.feature_importances_

array([1074, 1180, 1237,  525,  630,  677,  789,  696,  980,    0,  485,
          0,  233,  502,  745,  200,   57,  117,   96,  249,   78,  185,
          0,   90,    0,  150,   95,   97,   60,   62,   80,   42,   61,
         43,   78, 1359, 1005,  651,  443,  358,  384,  385,  358,  449,
        316,  371,  107,  106,   80,   72,   50,   45,   70,   98,  147,
        300,  275,  360,  169,   61,  106,  101,  114,   44,  177,    0,
         56,    0,  330,  284,  510,   96,   30,   65,   72,  124,   39,
         96,    0,   31,    0,  158,  198,  276,  203,  163,  142,  160,
        155,  184,  316,  774,  455,  431,  328,  292,  201,  220,  230,
        118,   62,  236,  122,  111,  143,  125,  104,   89,  121,  121,
        152, 3015, 1829, 1320, 1810, 1767,  713,  949, 1282, 1971,  515,
        507,  294,  612,   54,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  806,  842,  903,  326,  317,    0,    0,    0, 1411,
        569, 2295, 1164, 2341, 1577, 1452, 4082], d

In [43]:
qdres = {}
for i, dkey in enumerate(index):
    qdres[tuple(dkey)] = cres[i]

In [44]:
sqd = {}
qdres = {}
with open('./initial_data/sample.csv/sample.csv', 'r') as fin:
    sample = fin.readlines()
    print(len(sample))
    for i in range(1,len(sample)):
        sample[i] = sample[i][:-1]
        tr = sample[i].split(',')
        if tr[0] not in sqd.keys():
            sqd[tr[0]] = []
        else:
            sqd[tr[0]].append(tr[1])
        qdres[tuple(tr)] = cres[i-1]

403972


In [45]:
results = []
for key in sqd:
    res = {}
    for doc in sqd[key]:
        try:
            res[doc] = qdres[(key,doc)]
        except:
            continue
    sortres = sorted(res.items(), key = operator.itemgetter(1), reverse = True)
    for i in range(len(sortres)):
        results.append([key,sortres[i][0]])

In [46]:
with open('submit.csv', 'w') as fout:
    fout.write('QueryId,DocumentId\n')
    for result in results:
        fout.write(result[0] + ',' + result[1]+'\n')

In [34]:
#get feature importance
featslist = []
with open('feats.txt') as fin:
    for line in fin.readlines():
        featslist.append(line[line.find(' ') + 1:-1])
feats = {}
for i, fimp in enumerate(lgrank.feature_importances_):
    feats[featslist[i]] = fimp
sfeats = sorted(feats.items(), key = operator.itemgetter(1), reverse = True)
with open('featsimp.txt','w') as fout:
    for el in sfeats:
        fout.write(el[0] + '\t' + str(el[1])+'\n')